In [2]:
# very imp and useful util
import os

from dotenv import load_dotenv
load_dotenv()

os.environ["OPENAI_API_KEY"] = os.environ.get("OPENAI_API_KEY")

from IPython.display import Markdown
import textwrap


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [3]:
# useful util
# import sys
# import pkg_resources
# print("Python interpreter:", sys.executable)
# print("Packages:")
# for dist in pkg_resources.working_set:
#     print(dist)


In [4]:
import bs4
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.chat_models import ChatOpenAI
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

In [9]:
loader = WebBaseLoader(
    web_paths=("https://www.notion.so/Week-6-1-React-Deeper-Dive-581a481892fe4d3a8e0cb5360ce8d73e",),
    bs_kwargs=dict(
        # this is helps to parse the html given by the url to text
        parse_only=bs4.SoupStrainer(
            # class_=("post-content", "post-title", "post-header")
        )
    )
)


In [10]:
docs = loader.load()
docs

[Document(page_content='Notion – The all-in-one workspace for your notes, tasks, wikis, and databases.', metadata={'source': 'https://www.notion.so/Week-6-1-React-Deeper-Dive-581a481892fe4d3a8e0cb5360ce8d73e', 'title': 'Notion – The all-in-one workspace for your notes, tasks, wikis, and databases.', 'description': "A new tool that blends your everyday work apps into one. It's the all-in-one workspace for you and your team", 'language': 'No language found.'})]

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200, add_start_index=True)

In [ ]:
splits = text_splitter.split_documents(docs)

In [ ]:

vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

In [ ]:

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 4})

In [ ]:
# prompt with chat history memory
from langchain_core.prompts import PromptTemplate
template = """You are a chatbot having a conversation with a human your are a {ability}.

Given the following extracted parts of a long document and a question,
create a final answer using your intelligence.

(This context can be completely irrelevant to the question if so, skip it,
focus on giving the best answer to the human question)
{context}

{chat_history}
Human: {human_input}
Chatbot:"""

with_history_prompt = PromptTemplate.from_template(template)

In [ ]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.6)

In [ ]:
from langchain.memory import ConversationSummaryBufferMemory

memory = ConversationSummaryBufferMemory(
    memory_key="chat_history",
    input_key="human_input",
    output_key="AI",
    max_token_limit=100,
    llm=llm
)

In [ ]:
from operator import itemgetter
from langchain.chains import LLMChain

# print(memory.load_memory_variables("chat_history"))
custom_chain = (
    {
        "chat_history": itemgetter("chat_history") ,
        "ability": itemgetter("ability"),
        "human_input": itemgetter("human_input"),
        "context": itemgetter("input_documents")
    }
    # | with_history_prompt
    | LLMChain(
        llm=llm, 
        prompt=with_history_prompt,
        # verbose=True
    )
)


In [ ]:
question = "Can you emphasis more on COT?"

In [ ]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

docs = format_docs(retriever.invoke(question))

In [ ]:
# to_markdown(docs)

In [ ]:
response_data = custom_chain.invoke(
    {
        "input_documents":docs,
        "chat_history":memory.load_memory_variables("chat_history")['chat_history'],
        "human_input": question, 
        "ability": "Good talker"
    }
)
memory.save_context({"human_input":question},{"AI":response_data['text']})

In [ ]:
to_markdown(memory.load_memory_variables("chat_history")['chat_history']) # this is whats stored as history
to_markdown(response_data['text']) # ai response

In [ ]:
# memory.clear()